first we filter out obvious nsfw keywords

In [36]:
# Define a list of keywords that might indicate NSFW content (case-insensitive)
nsfw_keywords = [
    "nsfw",
    "porn",
    "sex",
    "xxx",
    "nude",
    "naked",
    "erotic",
    "kinky",
    "fetish",
    "bondage",
    "bdsm",
    "hentai",
    "yaoi",
    "yuri",
    "ecchi",
    "lewd",
    "lewding",
    "lewded",
    "lingerie",
    "underwear",
    "bikini",
    "swimsuit",
    "swimwear",
    "bra",
    "thong",
    "g-string",
    "breast",
    "boob",
    "nipple",
    "cleavage",
    "goddess",
    "sexy",
    "horny",
    "catsuit",
    "latex",
    "suggestive",
    "ass",
    "hardcore",
    "lesbian",
    "feminine"
]

# Read sentences from the file
unsafe_sentences = []
import re
pattern = re.compile(r"\b(?:{})\b".format("|".join(nsfw_keywords)), flags=re.IGNORECASE)
with open("../clip_interrogator/data/flavors.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        sentence = line.strip()
        # check if the sentence contains any NSFW keywords as whole words
        if pattern.search(sentence):
            unsafe_sentences.append(sentence)

# Save the filtered sentences to words.txt
with open("words.txt", "w", encoding="utf-8") as outfile:
    for sentence in unsafe_sentences:
        outfile.write(sentence + "\n")

In [3]:
import sys
sys.path.append('../clip_interrogator')

caption_model_name = 'blip-large' #@param ["blip-base", "blip-large", "git-large-coco"]
clip_model_name = 'ViT-L-14/openai' #@param ["ViT-L-14/openai", "ViT-H-14/laion2b_s32b_b79k"]

from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)

/home/kitegg/cert/office/clip-interrogator-sfw/ci_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading caption model blip-large...
Loading CLIP model ViT-L-14/openai...


/home/kitegg/cert/office/clip-interrogator-sfw/ci_env/lib/python3.11/site-packages/open_clip/factory.py:380: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Loaded CLIP model and data in 2.12 seconds.


In [6]:
import math
from typing import List
import torch
from tqdm import tqdm

def feature(text: str) -> torch.Tensor:
    text_tokens = ci.tokenize([text]).to(ci.device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = ci.clip_model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        return text_features


def similarity(self, image_features: torch.Tensor, lable_table:any, threshold: float = 0.8) -> List[tuple]:
        text_embeds = torch.stack([torch.from_numpy(t)
                                  for t in lable_table.embeds]).to(self.device)
        with torch.amp.autocast('cuda'):
            similarity = image_features @ text_embeds.T
                        
        sim_scores = similarity.float().cpu().squeeze(0)
        print("len of similarity", len(sim_scores))

        selected = (sim_scores > threshold).nonzero(as_tuple=True)[0]
        sorted_indices = selected[sim_scores[selected].argsort(descending=True)]
        indices = sorted_indices.tolist()
        results = [(lable_table.labels[i], sim_scores[i].item()) for i in indices]
        return results
        
        
        print(indices)
        
text_features = feature("Abraham")
sim = similarity(ci, text_features, ci.flavors, threshold=0.8)
print(sim)

len of similarity 100970
[('abraham teerlink', 0.86962890625), ('ambeint', 0.83642578125), ('abraham lincoln', 0.83544921875), ('adrian', 0.83203125), ('john', 0.83154296875), ('thomas', 0.8310546875), ('arthur', 0.82666015625), ('adam', 0.82421875), ('nicholas', 0.82275390625), ('jeremiah', 0.82080078125), ('christian', 0.82080078125), ('arham horror', 0.82080078125), ('john william', 0.8203125), ('sabbas', 0.81982421875), ('edward', 0.8193359375), ('george', 0.818359375), ('david a', 0.818359375), ('of an old man', 0.81787109375), ('walter', 0.81787109375), ('abram games', 0.81689453125), ('arian', 0.81689453125), ('american', 0.8154296875), ('andreas', 0.81494140625), ('american man', 0.81298828125), ('eddie adams', 0.8125), ('charles', 0.8115234375), ('johnny', 0.81103515625), ('edward robert', 0.810546875), ('raymond', 0.810546875), ('the man', 0.81005859375), ('arri', 0.8095703125), ('jeremy', 0.80908203125), ('jack', 0.80810546875), ('grand', 0.80810546875), ('abbeys', 0.8071289

/tmp/ipykernel_476052/2344754741.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


In [9]:
for sentence in nsfw_keywords:
    text_tokens = ci.tokenize([sentence]).to(ci.device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = ci.clip_model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    sim = similarity(ci, text_features, ci.flavors, threshold=0.8)
    # save to file called sentence + .txt
    with open(f"sim/{sentence}.txt", "w", encoding="utf-8") as outfile:
        for s in sim:
            outfile.write(f"{s[0]}\n")

/tmp/ipykernel_476052/3643669307.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970


In [37]:
import glob

all_entries = []
for txt_file in glob.glob("sim/*.txt"):
    with open(txt_file, "r", encoding="utf-8") as f:
        for line in f:
            all_entries.append(line.strip())

# add unsafe_sentences to all_entries
all_entries.extend(unsafe_sentences)

unique_entries = set(all_entries)

In [21]:
additional_entries = []
for sentence in unique_entries:
    text_tokens = ci.tokenize([sentence]).to(ci.device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = ci.clip_model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    sim = similarity(ci, text_features, ci.flavors, threshold=0.95)
    # save to file called sentence + .txt
    for s in sim:
        additional_entries.append(s[0])

/tmp/ipykernel_476052/1820745464.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970
len of similarity 100970


In [39]:
#len(additional_entries)
len(set(all_entries))

1303

In [31]:
# save additional_entries to file
with open("additional_entries.txt", "w", encoding="utf-8") as outfile:
    for sentence in set(additional_entries):
        outfile.write(sentence + "\n")

In [38]:
all_entries.extend(additional_entries)

unique_entries = set(all_entries)
with open("nsfw.txt", "w", encoding="utf-8") as outfile:
    for sentence in unique_entries:
        outfile.write(sentence + "\n")


In [40]:
with open("../clip_interrogator/data/flavors.txt", "r", encoding="utf-8") as infile:
    all_flavors = [line.strip() for line in infile]

flavors_sfw = [flavor for flavor in all_flavors if flavor not in unique_entries]

with open("flavors_sfw.txt", "w", encoding="utf-8") as outfile:
    for flavor in flavors_sfw:
        outfile.write(flavor + "\n")